# Iterativne metode


Za velike sustave, a posebno za sustave s malom ispunom (malo elemenata različitih od nule), te ukoliko je matrica sustava _strogo dijagonalno dominantna_, rješenje se može brzo naći _iterativnim metodama_
(vidi [Numerička matematika, poglavlje 3.8][RS04]):

__Definicija.__ Funkcija $F:\mathbb{R}^n\to \mathbb{R}^n$ je _kontrakcija_ ako postoji broj $q<1$ za koji vrijedi

$$
\| F(x)-F(y)\| < q\|x-y\|\qquad \forall x,y.
$$

__Banachov teorem o fiksnoj točki.__
Ako je $F$ kontrakcija, onda niz definiran s

$$ 
x_{k+1}=F(x_k)
$$

konvergira prema jedinstvenom vektoru $\tilde x$ za kojeg vrijedi

$$
\tilde x = F(\tilde x).
$$

$\tilde x$ se zove _fiksna točka_ funkcije $F$. Za pogrešku u $k$-tom koraku vrijede ocjene

$$
\|x_k- \tilde x\| \leq \frac{q}{1-q} \|x_k-x_{k-1}\|
$$

i 

$$
\|x_k- \tilde x\| \leq \frac{q^k}{1-q} \|x_1-x_{0}\|,
$$

pri čemu je druga ocjena bolja. Brzina konvergencije je _linearna_,

$$
\|x_{k+1}-\tilde x\| \leq q\| x_k-\tilde x\|.
$$

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, Osijek, 2004."


## Jacobijeva i Gauss-Seidelova metoda

Neka je 

$$F(x)=Bx+c,$$

pri čemu je $B$ kvadratna matrica. Tada je

$$
\| F(x)-F(y)\|=\| Bx+c-(By+c)\|=\|B(x-y)\| \leq \|B\| \|x-y\|,
$$

pa je $F$ kontrakcija ako je

$$
 \|B\|=q<1.
$$

Neka je zadan sustav  $Ax=b$. Matricu $A$ rastavimo kao

$$
A=D\,(L+I+U)
$$

pri čemu je $D$ dijagonalna matrica, $L$ strogo donje trokutasta matrica i $U$ strogo gornje trokutasta matrica.

### Jacobijeva metoda 

Neka je 

$$
B=-(L+U), \quad c=D^{-1}b.
$$


Ako je matrica $A$ _strogo dijagonalno dominantna_, 

$$
\| B\|_{\infty} = \max_i \sum_{{j=1} \atop {j\neq i}}^n \frac{|a_{ij}|}{|a_{ii}|}<1,
$$

onda je preslikavanje $F$ kontrakcija (moguće je uzeti i druge norme) pa niz

$$
x_{k+1}=-(L+U)x_k+c
$$

konvergira prema rješenju sustava $x$.

### Gauss-Seidel-ova metoda 

Neka je 

$$
B=-(I+L)^{-1}U, \quad c=(I+L)^{-1}\, D^{-1}b.
$$

Bez dokaza navodimo sljedeću tvrdnju: ako je matrica $A$ strogo dijagonalno dominantna,
onda je preslikavanje $F$ kontrakcija pa niz

$$
x_{k+1}=-(I+L)^{-1}Ux_k+(I+L)^{-1}D^{-1}b,
$$

odnosno

$$
x_{k+1}=-Lx_{k+1}-Ux_k+D^{-1}b,
$$

konvergira prema rješenju sustava $x$.

In [1]:
using LinearAlgebra
function myjacobi(A::Array,b::Array,x::Array)
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    B=-(L+U)
    c=b./D
    q=norm(B,Inf)
    # @show q
    while d>tol
        y=B*x+c
        d=norm(x-y,Inf)
        # @show d
        x=y
    end
    x,d
end

myjacobi (generic function with 1 method)

In [2]:
import Random
Random.seed!(123)
n=8
A=rand(n,n)
# Napravimo matricu dijagonalno dominantnom
A=A+n*I
b=rand(n)

8-element Array{Float64,1}:
 0.9249649192337484 
 0.34893902341348704
 0.7844226234383731 
 0.8497747226798917 
 0.11992936825521072
 0.8601428096736736 
 0.5768075478278709 
 0.17805867488523286

In [3]:
# Početni vektor
x0=rand(n)

8-element Array{Float64,1}:
 0.9519169935552045  
 0.8780997467517853  
 0.009166252824636123
 0.5958963124776646  
 0.9543508839651555  
 0.8419452294952618  
 0.2961019161242846  
 0.8557272304345225  

In [4]:
x,d=myjacobi(A,b,x0)
# Rješenje
x

8-element Array{Float64,1}:
  0.08980305378762726 
  0.014317578796243843
  0.07456867749469606 
  0.0821775198466023  
 -0.006931841179289444
  0.08111609269051698 
  0.056856588309506684
 -0.005698840205529316

In [5]:
# Norma razlike dvije zadnje iteracije
d

1.6361911825413245e-13

In [6]:
# Rezidual
r=A*x-b

8-element Array{Float64,1}:
 -3.3995029014022293e-13
 -4.472533454702443e-13 
 -3.397282455352979e-13 
 -3.3006930522105904e-13
 -3.97237798210881e-13  
 -5.331290964250002e-13 
 -2.73003841755326e-13  
 -4.266031972122164e-13 

In [7]:
# Provjerimo i normu relativnog reziduala
norm(r)/(norm(A)*norm(x))

2.6299688420966703e-13

In [8]:
function mygaussseidel(A::Array,b::Array,x::Array)
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    # B=-inv(I+L)*U
    B=-(I+L)\U
    c=(I+L)\(b./D)
    # @show norm(U,Inf)
    y=Vector{Float64}(undef,n)
    while d>tol
        y=B*x+c
        d=norm(x-y)
        x=y
    end
    x,d
end

mygaussseidel (generic function with 1 method)

In [9]:
x,d=mygaussseidel(A,b,x0)
# Rješenje
x

8-element Array{Float64,1}:
  0.08980305378765455  
  0.01431757879628013  
  0.07456867749472151  
  0.08217751984662743  
 -0.0069318411792575986
  0.08111609269056344  
  0.05685658830953137  
 -0.0056988402054908545

In [10]:
# Norma razlike dvije zadnje iteracije
d

7.735306213254282e-14

In [11]:
# Rezidual
A*x-b

8-element Array{Float64,1}:
 -5.88418203051333e-15  
 -2.930988785010413e-14 
 -2.6645352591003757e-14
 -1.7430501486614958e-14
 -6.7307270867900115e-15
 -3.6637359812630166e-15
 -1.1102230246251565e-16
  2.7755575615628914e-17

Izmjerimo brzinu za veće matrice:

In [12]:
n=1024
A=rand(n,n)
# Napravimo matricu dijagonalno dominantnom
A=A+n*I
b=rand(n)
# Pocetni vektor
x0=rand(n)

1024-element Array{Float64,1}:
 0.16641614093330026
 0.602302264828624  
 0.466840691287818  
 0.39403858534532366
 0.0791761643861233 
 0.13658772904085992
 0.6473994571097141 
 0.18238250664440225
 0.29197913832578726
 0.10273804645853324
 0.3907351461765771 
 0.8087040779127272 
 0.6398599234494022 
 ⋮                  
 0.05098896595273139
 0.26938136455069683
 0.9663749687160927 
 0.6569570021468696 
 0.6384179322464334 
 0.6936973328260423 
 0.5029705029470937 
 0.16629793407793225
 0.7268488402721949 
 0.18606528684826107
 0.7721857392292089 
 0.8819063375259617 

In [16]:
@time mygaussseidel(A,b,x0);

  0.025047 seconds (73 allocations: 64.413 MiB)


In [17]:
@time A\b;

  0.011337 seconds (9 allocations: 8.016 MiB)


__Zadatak:__ Probajte preraditi programe tako da alociraju manje memorije.